In [3]:
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from kb_python.utils import import_matrix_as_anndata
from sklearn.linear_model import LinearRegression
from upsetplot import from_contents, from_indicators
from upsetplot import plot as upset
from scipy.stats import ttest_ind_from_stats
import anndata 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mx.mx_inspect import mx_inspect_rows, mx_inspect_cols
from mx.mx_normalize import mx_normalize
import muon
import scanpy
from scipy.io import mmread

fsize = 15
import sys

def nd(arr):
    return np.asarray(arr).reshape(-1)


def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax.plot(lims, lims, c="k", alpha=0.75, zorder=0)
    ax.set(**{"aspect": "equal", "xlim": lims, "ylim": lims})
    return ax


plt.rcParams.update({"font.size": fsize})
%config InlineBackend.figure_format = 'retina'

# scATAK peaks, subsampled reads

In [2]:
a = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_peaks_subsam/counts_mult/adata.h5ad")
g = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_genes_subsam/counts_nolist/adata.h5ad") # multiome RNA whitelist
cm = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_genes_subsam/counts_mult/adata.h5ad")

# load whitelists
awl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/ATA-737K-arc-v1_rc.txt", header=None, names=["ata"]) # 737K-arc-v1_rc.txt
gwl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/RNA-737K-arc-v1.txt", header=None, names=["rna"]) # 10x_version3_whitelist.txt

# Map BCs between MOME-RNA and MOME-ATAC
bcmap = pd.concat([awl, gwl], axis=1).set_index("ata")

a.obs["ata_bcs"] = a.obs.index.values
a.obs["rna_bcs"] = a.obs.index.map(bcmap["rna"])
a.obs.set_index("rna_bcs", inplace=True)

common = np.intersect1d(a.obs.index.values, np.intersect1d(g.obs.index.values, cm.obs.index.values))

ma = a.copy() # a[common][a[common].obs.sort_index().index].copy()
mg = g.copy() # g[common][g[common].obs.sort_index().index].copy()
mg_cm = cm.copy() # cm[common][cm[common].obs.sort_index().index].copy()

In [3]:
# Barcodes
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
mg_cm.obs = mx_inspect_rows(mg_cm.X.copy(), mg_cm.obs.index.values)

# Features
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)
mg_cm.var = mx_inspect_cols(mg_cm.X.copy(), mg_cm.var.index.values)

In [4]:
# Make sure the barcodes are concordant between the umi counts and read counts
(mg.obs.index.values != mg_cm.obs.index.values).sum()

0

In [7]:
# Map the Cell Ranger Leiden clusters
gdf = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/download/analysis/clustering/gex/graphclust/clusters.csv")
adf = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/download/analysis/clustering/atac/graphclust/clusters.csv")

gdf["bcs"] = gdf["Barcode"].apply(lambda x: x.split("-")[0])
gdf = gdf.set_index("bcs")

adf["bcs"] = adf["Barcode"].apply(lambda x: x.split("-")[0])
adf = adf.set_index("bcs")

In [12]:
mg.obs["cluster"] = mg.obs.index.map(gdf["Cluster"])
mg.obs["custer"] = mg.obs["cluster"].fillna(-1).astype(int)

ma.obs["cluster"] = ma.obs.index.map(adf["Cluster"])
ma.obs["custer"] = ma.obs["cluster"].fillna(-1).astype(int)

In [24]:
mg.obs = pd.concat([mg.obs, 
                    mg_cm.obs.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

mg.var = pd.concat([mg.var, 
                    mg_cm.var.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

In [100]:
mu = muon.MuData({"gene": mg, "atac": ma})

In [26]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/kb_peaks_subsam.h5mu")

# Prep RNA (kb genes) and ATAC (scATAK peaks) Subsampled Matrices

In [94]:
gg = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/rna_x/out_kb_genes_subsam/counts_nolist/adata.h5ad") # standard 10xv3 whitelist
aa = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_kb_peaks_subsam/counts_mult/adata.h5ad")
gg_cm = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/rna_x/out_kb_genes_subsam/counts_mult/adata.h5ad")

In [95]:
(gg_cm.obs.index.values != gg.obs.index.values).sum()

0

In [96]:
# Barcodes
aa.obs = mx_inspect_rows(aa.X.copy(), aa.obs.index.values)
gg.obs = mx_inspect_rows(gg.X.copy(), gg.obs.index.values)
gg_cm.obs = mx_inspect_rows(gg_cm.X.copy(), gg_cm.obs.index.values)

# Features
aa.var = mx_inspect_cols(aa.X.copy(), aa.var.index.values)
gg.var = mx_inspect_cols(gg.X.copy(), gg.var.index.values)
gg_cm.var = mx_inspect_cols(gg_cm.X.copy(), gg_cm.var.index.values)

In [97]:
gg.obs = pd.concat([gg.obs, 
                    gg_cm.obs.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

gg.var = pd.concat([gg.var, 
                    gg_cm.var.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

In [98]:
mu = muon.MuData({"gene": gg})
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/rna_x/kb_genes_subsam.h5mu")

mu = muon.MuData({"atac": aa})
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/kb_genes_subsam.h5mu")

In [27]:
# gg.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/rna_x/kb_genes_subsam.h5ad")
# aa.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/kb_peaks_subsam.h5ad")

# Prep kb peaks, cr gene quantification full reads

In [ ]:
gene = {
    "input": {
        "fastqs" : [
            "",
            "",
            ""
        ],
        "onlist": "",
        "index": "",
        "t2g": "",
        "seqspec": "",
    },
    "output": {
        "matrix": "",
        "barcodes": "",
        "features": "",
        "anndata": ""
    }
}

atac = {
    "input": {
        "fastqs" : [
            "",
            "",
            ""
        ],
        "onlist": "",
        "index": "",
        "t2g": "",
        "seqspec": "",
        "markers": ""
    },
    "output": {
        "matrix_counts": "",
        "barcodes_counts": "",
        "genes_counts": "",
        "matrix_reads": "",
        "barcodes_reads": "",
        "genes_reads": "",
        "anndata_counts": "",
        "anndata_reads": ""
    }
}

In [72]:
a = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_peaks/counts_mult/adata.h5ad")
g = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_cr_gene/counts_unfiltered/adata.h5ad")
cm = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_cr_gene/counts_mult/adata.h5ad")

# load whitelists
awl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/ATA-737K-arc-v1_rc.txt", header=None, names=["ata"]) # 737K-arc-v1_rc.txt
gwl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/RNA-737K-arc-v1.txt", header=None, names=["rna"]) # 10x_version3_whitelist.txt

# Map BCs between MOME-RNA and MOME-ATAC
bcmap = pd.concat([awl, gwl], axis=1).set_index("ata")

a.obs["ata_bcs"] = a.obs.index.values
a.obs["rna_bcs"] = a.obs.index.map(bcmap["rna"])
a.obs.set_index("rna_bcs", inplace=True)

# common = np.intersect1d(a.obs.index.values, np.intersect1d(g.obs.index.values, cm.obs.index.values))
common = np.intersect1d(g.obs.index.values, cm.obs.index.values)

ma = a.copy() # a[common][a[common].obs.sort_index().index].copy()
mg = g[common][g[common].obs.sort_index().index].copy()
mg_cm = cm[common][cm[common].obs.sort_index().index].copy()

In [73]:
print(mg.shape, mg_cm.shape, sep="\n")

(666440, 36601)
(666440, 36601)


In [74]:
# Barcodes
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
mg_cm.obs = mx_inspect_rows(mg_cm.X.copy(), mg_cm.obs.index.values)

# Features
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)
mg_cm.var = mx_inspect_cols(mg_cm.X.copy(), mg_cm.var.index.values)

In [75]:
(mg.obs.index.values != mg_cm.obs.index.values).sum()

0

In [76]:
# Map the Cell Ranger Leiden clusters
gdf = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/download/analysis/clustering/gex/graphclust/clusters.csv")
adf = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/download/analysis/clustering/atac/graphclust/clusters.csv")

gdf["bcs"] = gdf["Barcode"].apply(lambda x: x.split("-")[0])
gdf = gdf.set_index("bcs")

adf["bcs"] = adf["Barcode"].apply(lambda x: x.split("-")[0])
adf = adf.set_index("bcs")

In [77]:
mg.obs["cluster"] = mg.obs.index.map(gdf["Cluster"])
mg.obs["custer"] = mg.obs["cluster"].fillna(-1).astype(int)

ma.obs["cluster"] = ma.obs.index.map(adf["Cluster"])
ma.obs["custer"] = ma.obs["cluster"].fillna(-1).astype(int)

In [78]:
mg.obs = pd.concat([mg.obs, 
                    mg_cm.obs.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

mg.var = pd.concat([mg.var, 
                    mg_cm.var.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

In [79]:
mu = muon.MuData({"gene": mg, "atac": ma})

In [80]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/kb_peaks_cr_genes_kb_quant.h5mu")

In [83]:
cra = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_peaks_cr_quant_subsam/outs/raw_feature_bc_matrix/peaks.h5ad")
crg = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_peaks_cr_quant_subsam/outs/raw_feature_bc_matrix/genes.h5ad")

In [87]:
# Barcodes
crg.obs = mx_inspect_rows(crg.X.copy(), crg.obs.index.values)
cra.obs = mx_inspect_rows(cra.X.copy(), cra.obs.index.values)

# Features
crg.var = mx_inspect_cols(crg.X.copy(), crg.var.index.values)
cra.var = mx_inspect_cols(cra.X.copy(), cra.var.index.values)

In [88]:
mu = muon.MuData({"gene": crg, "atac": cra})

In [89]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/kb_peaks_cr_genes_cr_quant_subsam.h5mu")

# Prep cr peaks, cr gene quantification full reads

In [ ]:
gene = {
    "input": {
        "fastqs" : [
            "",
            "",
            ""
        ],
        "onlist": "",
        "index": "",
        "t2g": "",
        "seqspec": "",
    },
    "output": {
        "matrix": "",
        "barcodes": "",
        "features": "",
        "anndata": ""
    }
}

atac = {
    "input": {
        "fastqs" : [
            "",
            "",
            ""
        ],
        "onlist": "",
        "index": "",
        "t2g": "",
        "seqspec": "",
    },
    "output": {
        "matrix_counts": "",
        "barcodes_counts": "",
        "genes_counts": "",
        "matrix_reads": "",
        "barcodes_reads": "",
        "genes_reads": "",
        "anndata_counts": "",
        "anndata_reads": ""
    }
}

In [12]:
a  = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_time_atac/counts_mult/adata.h5ad")
g  = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_time_gene/counts_unfiltered/adata.h5ad")
cm = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_time_gene/counts_mult/adata.h5ad")

# load whitelists
awl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/ATA-737K-arc-v1_rc.txt", header=None, names=["ata"]) # 737K-arc-v1_rc.txt
gwl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/RNA-737K-arc-v1.txt", header=None, names=["rna"]) # 10x_version3_whitelist.txt

# Map BCs between MOME-RNA and MOME-ATAC
bcmap = pd.concat([awl, gwl], axis=1).set_index("ata")

a.obs["ata_bcs"] = a.obs.index.values
a.obs["rna_bcs"] = a.obs.index.map(bcmap["rna"])
a.obs.set_index("rna_bcs", inplace=True)

# common = np.intersect1d(a.obs.index.values, np.intersect1d(g.obs.index.values, cm.obs.index.values))
common = np.intersect1d(g.obs.index.values, cm.obs.index.values)

ma = a.copy() # a[common][a[common].obs.sort_index().index].copy()
mg = g[common][g[common].obs.sort_index().index].copy()
mg_cm = cm[common][cm[common].obs.sort_index().index].copy()

In [13]:
print(mg.shape, mg_cm.shape, sep="\n")

(666440, 36601)
(666440, 36601)


In [14]:
# Barcodes
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
mg_cm.obs = mx_inspect_rows(mg_cm.X.copy(), mg_cm.obs.index.values)

# Features
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)
mg_cm.var = mx_inspect_cols(mg_cm.X.copy(), mg_cm.var.index.values)

In [15]:
(mg.obs.index.values != mg_cm.obs.index.values).sum()

0

In [16]:
# Map the Cell Ranger Leiden clusters
gdf = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/download/analysis/clustering/gex/graphclust/clusters.csv")
adf = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/download/analysis/clustering/atac/graphclust/clusters.csv")

gdf["bcs"] = gdf["Barcode"].apply(lambda x: x.split("-")[0])
gdf = gdf.set_index("bcs")

adf["bcs"] = adf["Barcode"].apply(lambda x: x.split("-")[0])
adf = adf.set_index("bcs")

In [17]:
mg.obs["cluster"] = mg.obs.index.map(gdf["Cluster"])
mg.obs["custer"] = mg.obs["cluster"].fillna(-1).astype(int)

ma.obs["cluster"] = ma.obs.index.map(adf["Cluster"])
ma.obs["custer"] = ma.obs["cluster"].fillna(-1).astype(int)

In [18]:
mg.obs = pd.concat([mg.obs, 
                    mg_cm.obs.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

mg.var = pd.concat([mg.var, 
                    mg_cm.var.rename(columns={"counts_min": "reads_min", 
                                               "counts_max": "reads_max",
                                               "counts_sum": "reads_sum",
                                               "counts_mean": "reads_mean",
                                               "counts_nnzero": "reads_nnzero",
                                               "counts_variance": "reads_variance"})], axis=1)

In [19]:
mu = muon.MuData({"gene": mg, "atac": ma})

In [21]:
mu

MuData object with n_obs × n_vars = 726767 × 148344
  var:	'counts_min', 'counts_max', 'counts_sum', 'counts_mean', 'counts_nnzero', 'counts_variance'
  2 modalities
    gene:	666440 x 36601
      obs:	'counts_min', 'counts_max', 'counts_sum', 'counts_mean', 'counts_nnzero', 'counts_variance', 'cluster', 'custer', 'reads_min', 'reads_max', 'reads_sum', 'reads_mean', 'reads_nnzero', 'reads_variance'
      var:	'counts_min', 'counts_max', 'counts_sum', 'counts_mean', 'counts_nnzero', 'counts_variance', 'reads_min', 'reads_max', 'reads_sum', 'reads_mean', 'reads_nnzero', 'reads_variance'
    atac:	647812 x 111743
      obs:	'counts_min', 'counts_max', 'counts_sum', 'counts_mean', 'counts_nnzero', 'counts_variance', 'cluster', 'custer'
      var:	'counts_min', 'counts_max', 'counts_sum', 'counts_mean', 'counts_nnzero', 'counts_variance'

In [20]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/cr_peaks_cr_genes_kb_quant.h5mu")

In [22]:
cra = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/cr/outs/raw_feature_bc_matrix/peaks.h5ad")
crg = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/cr/outs/raw_feature_bc_matrix/genes.h5ad")

In [23]:
# Barcodes
crg.obs = mx_inspect_rows(crg.X.copy(), crg.obs.index.values)
cra.obs = mx_inspect_rows(cra.X.copy(), cra.obs.index.values)

# Features
crg.var = mx_inspect_cols(crg.X.copy(), crg.var.index.values)
cra.var = mx_inspect_cols(cra.X.copy(), cra.var.index.values)

In [24]:
mu = muon.MuData({"gene": crg, "atac": cra})

In [25]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/cr_peaks_cr_genes_cr_quant.h5mu")

# SHARE-Seqv2-HGMM

In [105]:
ma = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/hgmm/out_atac/counts_mult/adata.h5ad")
mg = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/hgmm/out_gene/counts_unfiltered/adata.h5ad")
mg_cm = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/hgmm/out_gene/counts_mult/adata.h5ad")

In [106]:
print(
    ma.shape,
    mg.shape,
    mg_cm.shape,
    sep="\n"
)

(1406878, 81923)
(564394, 117674)
(564394, 117674)


In [107]:
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)
mg_cm.obs = mx_inspect_rows(mg_cm.X.copy(), mg_cm.obs.index.values)
mg_cm.var = mx_inspect_cols(mg_cm.X.copy(), mg_cm.var.index.values)

In [111]:
df_a_hg = mx_inspect_rows(ma.X[:,ma.var.index.str.contains("human_")].copy(), ma.obs.index.values)
df_a_mm = mx_inspect_rows(ma.X[:,ma.var.index.str.contains("mouse_")].copy(), ma.obs.index.values)

df_a_hg = df_a_hg.add_prefix('human_')
df_a_mm = df_a_mm.add_prefix('mouse_')

In [115]:
ma.obs = pd.concat([ma.obs.add_prefix("total_"), df_a_hg, df_a_mm], axis=1)

In [112]:
df_g_hg = mx_inspect_rows(mg.X[:,mg.var.index.str.contains("ENSG")].copy(), mg.obs.index.values)
df_g_mm = mx_inspect_rows(mg.X[:,mg.var.index.str.contains("ENSMUS")].copy(), mg.obs.index.values)

df_g_hg = df_g_hg.add_prefix('human_')
df_g_mm = df_g_mm.add_prefix('mouse_')

In [116]:
mg.obs = pd.concat([mg.obs.add_prefix("total_"), df_g_hg, df_g_mm], axis=1)

In [117]:
rnm = {
    "counts_min": "reads_min", 
    "counts_max": "reads_max",
    "counts_sum": "reads_sum",
    "counts_mean": "reads_mean",
    "counts_nnzero": "reads_nnzero",
    "counts_variance": "reads_variance"
}

In [118]:
df_cm_hg = mx_inspect_rows(mg_cm.X[:,mg_cm.var.index.str.contains("ENSG")].copy(), mg_cm.obs.index.values)
df_cm_mm = mx_inspect_rows(mg_cm.X[:,mg_cm.var.index.str.contains("ENSMUS")].copy(), mg_cm.obs.index.values)

df_cm_hg = df_cm_hg.rename(columns=rnm).add_prefix('human_')
df_cm_mm = df_cm_mm.rename(columns=rnm).add_prefix('mouse_')

In [120]:
mg_cm.obs = pd.concat([mg_cm.obs.rename(columns=rnm).add_prefix("total_"), df_cm_hg, df_cm_mm], axis=1)

In [121]:
mg.obs = pd.concat([mg.obs, mg_cm.obs], axis=1)
mg.var = pd.concat([mg.var, mg_cm.var.rename(columns=rnm)], axis=1)

In [123]:
mu = muon.MuData({"gene": mg, "atac": ma})

In [127]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/shareseqv2/hgmm/mu.h5mu")

# ISSAAC-Seq HGMM

In [131]:
ma = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/issaacseq/hgmm/out_atac_subsam/counts_mult/adata.h5ad")
mg = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/issaacseq/hgmm/out_gene_subsam/counts_unfiltered/adata.h5ad")
mg_cm = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/issaacseq/hgmm/out_gene_subsam/counts_mult/adata.h5ad")

In [132]:
print(
    ma.shape,
    mg.shape,
    mg_cm.shape,
    sep="\n"
)

(243474, 160977)
(178832, 117674)
(178832, 117674)


In [133]:
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)
mg_cm.obs = mx_inspect_rows(mg_cm.X.copy(), mg_cm.obs.index.values)
mg_cm.var = mx_inspect_cols(mg_cm.X.copy(), mg_cm.var.index.values)

In [134]:
df_a_hg = mx_inspect_rows(ma.X[:,ma.var.index.str.contains("human_")].copy(), ma.obs.index.values)
df_a_mm = mx_inspect_rows(ma.X[:,ma.var.index.str.contains("mouse_")].copy(), ma.obs.index.values)

df_a_hg = df_a_hg.add_prefix('human_')
df_a_mm = df_a_mm.add_prefix('mouse_')

In [135]:
ma.obs = pd.concat([ma.obs.add_prefix("total_"), df_a_hg, df_a_mm], axis=1)

In [136]:
df_g_hg = mx_inspect_rows(mg.X[:,mg.var.index.str.contains("ENSG")].copy(), mg.obs.index.values)
df_g_mm = mx_inspect_rows(mg.X[:,mg.var.index.str.contains("ENSMUS")].copy(), mg.obs.index.values)

df_g_hg = df_g_hg.add_prefix('human_')
df_g_mm = df_g_mm.add_prefix('mouse_')

In [137]:
mg.obs = pd.concat([mg.obs.add_prefix("total_"), df_g_hg, df_g_mm], axis=1)

In [138]:
rnm = {
    "counts_min": "reads_min", 
    "counts_max": "reads_max",
    "counts_sum": "reads_sum",
    "counts_mean": "reads_mean",
    "counts_nnzero": "reads_nnzero",
    "counts_variance": "reads_variance"
}

In [139]:
df_cm_hg = mx_inspect_rows(mg_cm.X[:,mg_cm.var.index.str.contains("ENSG")].copy(), mg_cm.obs.index.values)
df_cm_mm = mx_inspect_rows(mg_cm.X[:,mg_cm.var.index.str.contains("ENSMUS")].copy(), mg_cm.obs.index.values)

df_cm_hg = df_cm_hg.rename(columns=rnm).add_prefix('human_')
df_cm_mm = df_cm_mm.rename(columns=rnm).add_prefix('mouse_')

In [140]:
mg_cm.obs = pd.concat([mg_cm.obs.rename(columns=rnm).add_prefix("total_"), df_cm_hg, df_cm_mm], axis=1)

In [141]:
mg.obs = pd.concat([mg.obs, mg_cm.obs], axis=1)
mg.var = pd.concat([mg.var, mg_cm.var.rename(columns=rnm)], axis=1)

In [142]:
mu = muon.MuData({"gene": mg, "atac": ma})

In [143]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/issaacseq/hgmm/mu.h5mu")

# Spatial

In [7]:
adata = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/spatial/out_kb_peaks/counts_mult/adata.h5ad")
adata.obs = mx_inspect_rows(adata.X.copy(), adata.obs.index.values)
adata.var = mx_inspect_cols(adata.X.copy(), adata.var.index.values)

In [8]:
bcs = pd.read_csv(
    "/home/sina/projects/atac/scATAK/data/spatial/spatial_barcodes.txt", 
    header=None, names=["bcs", "row", "col"], index_col=0, sep="\t"
)

In [9]:
adata.obs["row"] = adata.obs.index.map(bcs["row"]).values
adata.obs["col"] = adata.obs.index.map(bcs["col"]).values

In [10]:
mu = muon.MuData({"atac": adata,})

In [11]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/spatial/spatial.h5mu")

# SHARE-Seq BMMC

In [151]:
ma = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_atac/counts_mult/adata.h5ad")
mg = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_gene_subsam/counts_unfiltered/adata.h5ad")

In [155]:
dfa = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/geo_data/tbl.atac", sep="\t", index_col=0)
dfg = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/geo_data/tbl.rna", sep="\t", index_col=0)

common = np.intersect1d(dfa.index, dfg.index)

In [152]:
# Barcodes
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)

# Features
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)

In [158]:
# get the clusters for the filtered cells
fmg = mg[common].copy()
fmg.layers["raw"] = fmg.X.copy()
fmg.layers["log1pPF"] = mx_normalize(fmg.X.copy(), method="log1pPF")
fmg.X = fmg.layers["log1pPF"]

In [160]:
scanpy.pp.neighbors(fmg)
scanpy.tl.leiden(fmg)

         Falling back to preprocessing with `sc.pp.pca` and default params.


In [177]:
# map leiden to original mg object
mg.obs["leiden"] = mg.obs.index.map(fmg.obs["leiden"])
mg.obs["leiden"] = mg.obs["leiden"].astype(float).fillna(-1).astype(int)

In [179]:
mg.obs["pass_filter"] = mg.obs.index.isin(common)
ma.obs["pass_filter"] = ma.obs.index.isin(common)

In [180]:
mu = muon.MuData({"gene": mg, "atac": ma})

In [154]:
mu.write_h5mu("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/mu.h5mu")

In [ ]:
# write gene clustering
fmg.obs.to_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/data/bmmc/out_gene/clustering.txt", sep="\t", header=None)

# MOMEX SNPs

### ATAC SNPs

In [184]:
ref           = mmread("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_atac/counts_mult/ref.mtx").tocsr()
alt           = mmread("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_atac/counts_mult/alt.mtx").tocsr()
barcodes = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_atac/counts_mult/cells_x_genes.barcodes.txt", header=None, names=["bcs"], index_col=0)
genes    = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_atac/counts_mult/ref.genes.txt", header=None, names=["genes"], index_col=0)

M = ref+alt

# make anndata of ref, alt, M (sum of ref+alt)
ma = anndata.AnnData(X=M, obs=barcodes, var=genes, dtype=int)
ma.layers["ref"] = ref
ma.layers["alt"] = alt

# get rid of the "_ref"
ma.var.index = ma.var.index.map(lambda x: x.split("_")[0])

# do the genotyping
e = 1e-4 # 0.0001
a = ref*np.log(1-e) + alt*np.log(e) # hom_alt (1 in sparse matrix)
b = (ref+alt)*np.log(0.5)           # het     (2 in sparse matrix)
c = ref*np.log(e) + alt*np.log(1-e) # hom_ref (3 in sparse matrix)
ll = np.array([a.data, b.data, c.data])

geno = a.copy()
amax = np.argmax(ll, axis=0)+1
geno.data = amax # since its a sparse matrix andthe zeros are actually informative

ma.layers["gt"] = geno
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)

In [186]:
# load whitelists
awl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/ATA-737K-arc-v1_rc.txt", header=None, names=["ata"]) # 737K-arc-v1_rc.txt
gwl = pd.read_csv("/home/sina/projects/atac/scATAK/references/onlists/10xMOME/RNA-737K-arc-v1.txt", header=None, names=["rna"]) # 10x_version3_whitelist.txt

# Map BCs between MOME-RNA and MOME-ATAC
bcmap = pd.concat([awl, gwl], axis=1).set_index("ata")

ma.obs["ata_bcs"] = ma.obs.index.values
ma.obs["rna_bcs"] = ma.obs.index.map(bcmap["rna"])
ma.obs.set_index("rna_bcs", inplace=True)

### GENE SNPs

In [185]:
ref           = mmread("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_gene/counts_unfiltered/ref.mtx").tocsr()
alt           = mmread("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_gene/counts_unfiltered/alt.mtx").tocsr()
barcodes = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_gene/counts_unfiltered/cells_x_genes.barcodes.txt", header=None, names=["bcs"], index_col=0)
genes    = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_snps_gene/counts_unfiltered/ref.genes.txt", header=None, names=["genes"], index_col=0)

M = ref+alt

# make anndata of ref, alt, M (sum of ref+alt)
mg = anndata.AnnData(X=M, obs=barcodes, var=genes, dtype=int)
mg.layers["ref"] = ref
mg.layers["alt"] = alt

# get rid of the "_ref"
mg.var.index = mg.var.index.map(lambda x: x.split("_")[0])

# do the genotyping
e = 1e-4 # 0.0001
a = ref*np.log(1-e) + alt*np.log(e) # hom_alt (1 in sparse matrix)
b = (ref+alt)*np.log(0.5)           # het     (2 in sparse matrix)
c = ref*np.log(e) + alt*np.log(1-e) # hom_ref (3 in sparse matrix)
ll = np.array([a.data, b.data, c.data])

geno = a.copy()
amax = np.argmax(ll, axis=0)+1
geno.data = amax # since its a sparse matrix andthe zeros are actually informative

mg.layers["gt"] = geno
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)

In [187]:
ma.obs["cluster"] = ma.obs.index.map(adf["Cluster"])
mg.obs["cluster"] = mg.obs.index.map(gdf["Cluster"])

In [213]:
mg.var["tmp"] = 0

In [188]:
mg = mg[mg.obs.dropna().sort_index().index].copy()
ma = ma[ma.obs.dropna().sort_index().index].copy()

mg.obs["cluster"] = mg.obs["cluster"].astype(int)
ma.obs["cluster"] = ma.obs["cluster"].astype(int)

In [217]:
snp_mask = np.logical_and(mg.var.counts_nnzero > 4, 
                          ma.var.counts_nnzero > 4)

In [218]:
fmg = mg[:,snp_mask].copy()
fma = ma[:,snp_mask].copy()

In [219]:
print(fma.shape, fmg.shape,sep="\n")

(10970, 73976)
(10970, 73976)


In [225]:
fma.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/atac.h5ad")
fmg.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/gene.h5ad")

In [224]:
# write anndata instead of muon for snps

# SHARE-Seqv2 BMMC SNPs

In [232]:
cl = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_gene/clustering.txt", sep="\t", header=None, names=["bcs", "cluster"], index_col=0)

### ATAC SNPs

In [227]:
ref           = mmread("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_atac/counts_mult/ref.mtx").tocsr()
alt           = mmread("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_atac/counts_mult/alt.mtx").tocsr()
barcodes = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_atac/counts_mult/cells_x_genes.barcodes.txt", header=None, names=["bcs"], index_col=0)
genes    = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_atac/counts_mult/ref.genes.txt", header=None, names=["genes"], index_col=0)

M = ref+alt

# make anndata of ref, alt, M (sum of ref+alt)
ma = anndata.AnnData(X=M, obs=barcodes, var=genes, dtype=int)
ma.layers["ref"] = ref
ma.layers["alt"] = alt

# get rid of the "_ref"
ma.var.index = ma.var.index.map(lambda x: x.split("_")[0])

# do the genotyping
e = 1e-4 # 0.0001
a = ref*np.log(1-e) + alt*np.log(e) # hom_alt (1 in sparse matrix)
b = (ref+alt)*np.log(0.5)           # het     (2 in sparse matrix)
c = ref*np.log(e) + alt*np.log(1-e) # hom_ref (3 in sparse matrix)
ll = np.array([a.data, b.data, c.data])

geno = a.copy()
amax = np.argmax(ll, axis=0)+1
geno.data = amax # since its a sparse matrix andthe zeros are actually informative

ma.layers["gt"] = geno
ma.obs = mx_inspect_rows(ma.X.copy(), ma.obs.index.values)
ma.var = mx_inspect_cols(ma.X.copy(), ma.var.index.values)

In [228]:
ref           = mmread("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_gene/counts_unfiltered/ref.mtx").tocsr()
alt           = mmread("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_gene/counts_unfiltered/alt.mtx").tocsr()
barcodes = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_gene/counts_unfiltered/cells_x_genes.barcodes.txt", header=None, names=["bcs"], index_col=0)
genes    = pd.read_csv("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/out_snp_gene/counts_unfiltered/ref.genes.txt", header=None, names=["genes"], index_col=0)

M = ref+alt

# make anndata of ref, alt, M (sum of ref+alt)
mg = anndata.AnnData(X=M, obs=barcodes, var=genes, dtype=int)
mg.layers["ref"] = ref
mg.layers["alt"] = alt

# get rid of the "_ref"
mg.var.index = mg.var.index.map(lambda x: x.split("_")[0])

# do the genotyping
e = 1e-4 # 0.0001
a = ref*np.log(1-e) + alt*np.log(e) # hom_alt (1 in sparse matrix)
b = (ref+alt)*np.log(0.5)           # het     (2 in sparse matrix)
c = ref*np.log(e) + alt*np.log(1-e) # hom_ref (3 in sparse matrix)
ll = np.array([a.data, b.data, c.data])

geno = a.copy()
amax = np.argmax(ll, axis=0)+1
geno.data = amax # since its a sparse matrix andthe zeros are actually informative

mg.layers["gt"] = geno
mg.obs = mx_inspect_rows(mg.X.copy(), mg.obs.index.values)
mg.var = mx_inspect_cols(mg.X.copy(), mg.var.index.values)

In [233]:
ma.obs["cluster"] = ma.obs.index.map(cl["cluster"])
mg.obs["cluster"] = mg.obs.index.map(cl["cluster"])

In [234]:
mg = mg[mg.obs.dropna().sort_index().index].copy()
ma = ma[ma.obs.dropna().sort_index().index].copy()

mg.obs["cluster"] = mg.obs["cluster"].astype(int)
ma.obs["cluster"] = ma.obs["cluster"].astype(int)

In [235]:
snp_mask = np.logical_and(mg.var.counts_nnzero > 6, 
                          ma.var.counts_nnzero > 6)

In [236]:
fmg = mg[:,snp_mask].copy()
fma = ma[:,snp_mask].copy()

In [237]:
print(fma.shape, fmg.shape, sep="\n")

(48559, 67867)
(48559, 67867)


In [238]:
fma.write_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/atac.h5ad")
fmg.write_h5ad("/home/sina/projects/atac/scATAK/data/shareseqv2/bmmc/gene.h5ad")

# snATAK Cell Ranger comparison

### cmp kb cr on cr peaks

In [253]:
def rc(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    reverse_complement = "".join(complement.get(base, base) for base in reversed(seq))
    return reverse_complement

## load in matrices

# Cellranger
p = "/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_cr_peaks_cr_quant/quant-cr/outs/raw_peak_bc_matrix/adata.h5ad"
crcr = anndata.read_h5ad(p)

# This dataset had revc barcodes, so correct
crcr.obs.index = crcr.obs.index.map(rc)

# These are the barcodes that pass filter
fbcs = pd.read_csv("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_cr_peaks_cr_quant/quant-cr/outs/filtered_peak_bc_matrix/barcodes.txt", header=None, names=["bcs"])["bcs"].apply(rc).sort_values().values
p = "/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_cr_peaks_kb_quant/quant-kb/counts_mult/adata.h5ad"
kbcr = anndata.read_h5ad(p)

# Get the common barcodes
kbcrbcs = kbcr.obs.index.values
crcrbcs = crcr.obs.index.values

common_bcs = np.intersect1d(kbcrbcs, crcrbcs)

# Make raw matrices on common barcodes
raw_crcr = crcr[common_bcs].copy()
raw_kbcr = kbcr[common_bcs].copy()

# Make filtered matrices on common barcodes
crcr = raw_crcr[fbcs].copy()
kbcr = raw_kbcr[fbcs].copy()

In [254]:
raw = muon.MuData({"cr": raw_crcr, "kb": raw_kbcr})
fil = muon.MuData({"cr": crcr, "kb": kbcr})

In [255]:
raw.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/raw_cr_peaks.h5mu")
fil.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/fil_cr_peaks.h5mu")

### cmp kb cr on kb peaks

In [256]:
# load in matrices

# Cellranger
p = "/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_kb_peaks_cr_quant/cr-peaks/outs/raw_peak_bc_matrix/adata.h5ad"
crkb = anndata.read_h5ad(p)

# This dataset had revc barcodes, so correct
crkb.obs.index = crkb.obs.index.map(rc)

# These are the barcodes that pass filter
fbcs = pd.read_csv(
    "/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_kb_peaks_cr_quant/cr-peaks/outs/filtered_peak_bc_matrix/barcodes.txt", 
    header=None, names=["bcs"])["bcs"].apply(rc).sort_values().values

p = "/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_kb_peaks_kb_quant/kb-peaks/counts_mult/adata.h5ad"
kbkb = anndata.read_h5ad(p)

# Get the common barcodes
kbkbbcs = kbkb.obs.index.values
crkbbcs = crkb.obs.index.values

common_bcs = np.intersect1d(
    kbkbbcs, 
    crkbbcs
)



# Make raw matrices on common barcodes
raw_crkb = crkb[common_bcs].copy()
raw_kbkb = kbkb[common_bcs].copy()

crkb_peaks = raw_crkb.var.index.values
kbkb_peaks = raw_kbkb.var.index.values
common_peaks = np.intersect1d(
    kbkb_peaks, 
    crkb_peaks
)

# Make raw matrices on common barcodes
raw_crkb = raw_crkb[:,common_peaks].copy()
raw_kbkb = raw_kbkb[:,common_peaks].copy()

# Make filtered matrices on common barcodes
crkb = raw_crkb[fbcs].copy()
kbkb = raw_kbkb[fbcs].copy()

In [ ]:
# write anndata

In [257]:
raw = muon.MuData({"cr": raw_crkb, "kb": raw_kbkb})
fil = muon.MuData({"cr": crkb, "kb": kbkb})

In [258]:
raw.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/raw_kb_peaks.h5mu")
fil.write_h5mu("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/fil_kb_peaks.h5mu")

# 10x tech comparison

In [4]:
acc = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_c/out_kb_peaks_subsam/counts_mult/adata.h5ad")
acx = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_kb_peaks_subsam/counts_mult/adata.h5ad")
mcc = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_c/out_kb_peaks_subsam/counts_mult/adata.h5ad")
mcx = anndata.read_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_peaks_subsam/counts_mult/adata.h5ad")

In [5]:
acc.obs = mx_inspect_rows(acc.X.copy(), acc.obs.index.values)
acx.obs = mx_inspect_rows(acx.X.copy(), acx.obs.index.values)
mcc.obs = mx_inspect_rows(mcc.X.copy(), mcc.obs.index.values)
mcx.obs = mx_inspect_rows(mcx.X.copy(), mcx.obs.index.values)

acc.var = mx_inspect_cols(acc.X.copy(), acc.var.index.values)
acx.var = mx_inspect_cols(acx.X.copy(), acx.var.index.values)
mcc.var = mx_inspect_cols(mcc.X.copy(), mcc.var.index.values)
mcx.var = mx_inspect_cols(mcx.X.copy(), mcx.var.index.values)

In [6]:
acc.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_c/out_kb_peaks_subsam/adata.h5ad")
acx.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/atac_x/out_kb_peaks_subsam/adata.h5ad")
mcc.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_c/out_kb_peaks_subsam/adata.h5ad")
mcx.write_h5ad("/home/sina/projects/atac/scATAK/data/10xPBMC/mome_x/out_kb_peaks_subsam/adata.h5ad")